EECS 542 Final Project - Swin Transformer

Jiangyan Feng, Pengfei Gao, Yilin Li, Zekun Li

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import einops
from timm.models.layers import to_2tuple
! pip install einops
! pip install timm

1. Patch partition, Linear embedding, Patch merging (Zekun Li)

In [ ]:
class PatchEmbed(nn.Module):
  def __init__(self, patch_size, in_c, output_dim, norm=None):
    self.linear = nn.Conv2d(in_c, output_dim, kernel_size=patch_size, stride=patch_size, padding=0, dtype=x.dtype) # C -> output_dim, or use in_chans
    self.norm = norm
  def forward(self, x, patch_size): ## x: B * C * H * W
    B, C, H, W = x.shape
    if (H % patch_size) or (W % patch_size):
      x = F.pad(x, (0, patch_size - W % patch_size, 0, patch_size - H % patch_size, 0, 0))
    x = self.linear(x)
    x = x.flatten(2)
    x = x.transpose(1, 2)
    if self.norm: x = self.norm(x)
    return x 

class PatchMerging(nn.Module):
  def __init__(self, in_c, norm=nn.LayerNorm):
    self.norm = norm
    self.linear = nn.Linear(in_c * 4, in_c * 2, bias=False)
  def forward(self, x, H, W): ## x: B * L * C
    B, _, C = x.shape
    x = x.view(B, H, W, C)
    if H % 2 or W % 2:
      x = F.pad(x, (0, 0, 0, W % 2, 0, H % 2))
    x = torch.cat([x[:, 0::2, 0::2, :], x[:, 1::2, 0::2, :],
                  x[:, 0::2, 1::2, :], x[:, 1::2, 1::2, :]], -1).view(B, -1, C * 4)
    # norm = nn.LayerNorm(C * 4)
    # linear = nn.Linear(C * 4, C * 2, bias=False)
    return self.linear(self.norm(x))  ## x: B * HW/4 * 2C

2. Swin transformer block basic function (Yilin Li)

In [ ]:
class Mlp(nn.Module):
    """
    a 2-layer MLP with GELU nonlinearity activation function

    Args:
        in_features: input dimension
        hidden_features: hidden layer dimension
        out_features: output dimension
        act_layer: type of activation function(here we use GELU nonlinearity)
        drop: (dropout layer parameter) probability of an element to be zeroed
    """
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        """
        Args:
            x: input features
        """
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

def window_partition(x, window_size):
    """
    Args:
        x: (B, H, W, C)
        window_size (int): shape of window is (window_size * window_size)

    Returns:
        windows: (num_windows*B, window_size, window_size, C)
    """
    B, H, W, C = x.shape
    patch_num_y = H // window_size
    patch_num_x = W // window_size
    x = torch.reshape(x, (-1, patch_num_y, window_size, patch_num_x, window_size, C))
    x = x.permute(0, 1, 3, 2, 4, 5)
    windows = torch.reshape(x, (-1, window_size, window_size, C))
    return windows

def window_reverse(windows, window_size, H, W):
    """
    Args:
        windows: (num_windows*B, window_size, window_size, C)
        window_size (int): shape of window is (window_size * window_size)
        H (int): height of image
        W (int): width of image

    Returns:
        x: (B, H, W, C)
    """
    patch_num_y = H // window_size
    patch_num_x = W // window_size
    _, _, _, C = windows.shape
    x = torch.reshape(x, (-1, patch_num_y, patch_num_x, window_size, window_size, C))
    x = x.permute(0, 1, 3, 2, 4, 5)
    x = torch.reshape(x, (-1, H, W, C))
    return x

class DropPath(nn.Module):
    """
    Drop paths (Stochastic Depth) per sample  (when applied in main path of residual blocks).

    Args:
        drop_prob (float): probability of drop 
        scale_by_keep (bool): 
    """
    def __init__(self, drop_prob=None, scale_by_keep=True):
        super().__init__()
        self.drop_prob = drop_prob
        self.scale_by_keep = scale_by_keep

    def forward(self, x):
        """
        Args:
            x: input
        """
        if self.drop_prob == 0. or not self.training:
            return x
        keep_prob = 1 - self.drop_prob
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)
        random_tensor = x.new_empty(shape).bernoulli_(keep_prob)
        if keep_prob > 0.0 and self.scale_by_keep:
            random_tensor.div_(keep_prob)
        return x * random_tensor

def relative_position(window_size):
    """
    get pair-wise relative position index for each token inside the window

    Args:
    window_size (tuple[int]): The height and width of the window.
    
    Returns:
    relative_position_index (Wh*Ww, Wh*Ww)
    """
    coords_h = torch.arange(window_size[0])
    coords_w = torch.arange(window_size[1])
    coords = torch.stack(torch.meshgrid([coords_h, coords_w]))  # 2, Wh, Ww
    coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
    relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww
    relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
    relative_coords[:, :, 0] += window_size[0] - 1  # shift to start from 0
    relative_coords[:, :, 1] += window_size[1] - 1
    relative_coords[:, :, 0] *= 2 * window_size[1] - 1
    relative_position_index = relative_coords.sum(-1)  # Wh*Ww, Wh*Ww
    return relative_position_index

def trunc_normal(tensor, mean=0., std=1., a=-2., b=2.):
    def norm_cdf(x):
        # Computes standard normal cumulative distribution function
        return (1. + math.erf(x / math.sqrt(2.))) / 2.

    with torch.no_grad():
        # Values are generated by using a truncated uniform distribution and
        # then using the inverse CDF for the normal distribution.
        # Get upper and lower cdf values
        l = norm_cdf((a - mean) / std)
        u = norm_cdf((b - mean) / std)
        # Uniformly fill tensor with values from [l, u], then translate to
        # [2l-1, 2u-1].
        tensor.uniform_(2 * l - 1, 2 * u - 1)
        # Use inverse cdf transform for normal distribution to get truncated
        # standard normal
        tensor.erfinv_()
        # Transform to proper mean, std
        tensor.mul_(std * math.sqrt(2.))
        tensor.add_(mean)
        # Clamp to ensure it's in the proper range
        tensor.clamp_(min=a, max=b)
        return tensor

class WindowAttention(nn.Module):
    """ W-MSA / SW-MSA

    Args:
        dim (int): Number of input channels.
        window_size (tuple[int]): The height and width of the window.
        num_heads (int): Number of attention heads.
        qkv_bias (bool, optional):  If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set
        attn_drop (float, optional): Dropout ratio of attention weight. Default: 0.0
        proj_drop (float, optional): Dropout ratio of output. Default: 0.0
    """

    def __init__(self, dim, window_size, num_heads, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):

        super().__init__()
        self.dim = dim
        self.window_size = window_size  
        self.num_heads = num_heads
        self.scale = qk_scale or (dim // num_heads) ** (-0.5)

        # define a parameter table of relative position bias
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros((2 * window_size[0] - 1) * (2 * window_size[1] - 1), num_heads)) 

        self.register_buffer("relative_position_index", relative_position(self.window_size))

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        trunc_normal(self.relative_position_bias_table, std=.02)
        
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask=None):
        """
        Args:
            x: input features with shape of (num_windows*B, N, C)
            mask: (0/-inf) mask with shape of (num_windows, Wh*Ww, Wh*Ww) or None
        """
        B_, N, C = x.shape
        qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]  # make torchscript happy (cannot use tensor as tuple)

        q = q * self.scale
        attn = (q @ k.transpose(-2, -1))

        relative_position_bias = self.relative_position_bias_table[self.relative_position_index.view(-1)].view(
            self.window_size[0] * self.window_size[1], self.window_size[0] * self.window_size[1], -1)  
        relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()  
        attn = attn + relative_position_bias.unsqueeze(0)

        if mask is not None:
            nW = mask.shape[0]
            attn = attn.view(B_ // nW, nW, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
            attn = attn.view(-1, self.num_heads, N, N)
            attn = self.softmax(attn)
        else:
            attn = self.softmax(attn)

        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


NameError: ignored

3. Model construction (Pengfei Gao)

In [ ]:
def get_mask(input_resolution, window_size, shift_size):
  img_mask = torch.zeros((1, input_resolution[0], input_resolution[1], 1))
  h_slices = (slice(0, -window_size), slice(-window_size, -shift_size), slice(-shift_size, None))
  w_slices = (slice(0, -window_size), slice(-window_size, -shift_size), slice(-shift_size, None))
  count = 0
  for h in h_slices:
      for w in w_slices:
          img_mask[:, h, w, :] = count
          count += 1
  windows = window_partition(img_mask, window_size).view(-1, window_size ** 2)
  mask = windows.unsqueeze(1) - windows.unsqueeze(2)
  mask = attn_mask.masked_fill(mask != 0, float(-100.00)).masked_fill(mask == 0, float(0.00))
  return mask

class SwinTransformerBlock(nn.Module):
  def __init__(self, dim, input_resolution, num_heads, window_size=7, shift_size=0,
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0., drop_path=0.):
    super(SwinTransformerBlock, self).__init__()
    self.dim = dim
    self.input_resolution = input_resolution
    self.window_size = window_size
    self.shift_size = shift_size
    self.attention_mask = get_mask(input_resolution, window_size, shift_size)
    self.num_heads = num_heads
    #LayerNorm
    self.LN1 = nn.LayerNorm(dim)
    self.LN2 = nn.LayerNorm(dim)
    self.LN3 = nn.LayerNorm(dim)
    self.LN4 = nn.LayerNorm(dim)
    #MLP
    self.MLP1 = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)
    self.MLP2 = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)
    #WindowAttention
    self.WindowAttention1 = WindowAttention(dim, window_size=to_2tuple(self.window_size), num_heads=num_heads,
            qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)
    self.WindowAttention2 = WindowAttention(dim, window_size=to_2tuple(self.window_size), num_heads=num_heads,
            qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)
    #Drop Path
    self.drop_path = DropPath(drop_path if drop_path > 0. else 0.)
  def forward(self,x):
    H, W = self.input_resolution
    #first block
    B, L, C = x.shape

    x_initial = x
    x = self.LN1(x).reshape(B,H,W,C)
    # partition windows
    x_windows = window_partition(x, self.window_size)
    x_windows = x_windows.reshape(-1, self.window_size ** 2, C)
    # W-MSA
    WindowAttention1 = self.WindowAttention1(x_windows, mask=None)
    # merge windows
    WindowAttention1 = WindowAttention1.reshape(-1, self.window_size, self.window_size, C)
    x = window_reverse(WindowAttention1, self.window_size, H, W)
    x = x.reshape(B, H * W, C)
    #FFN
    x = initial_x + self.drop_path(x)
    initial_x = x
    x = self.LN2(x)
    x = self.MLP1(x)
    x = self.drop_path(x)
    x += initial_x

    #second block
    B, L, C = x.shape

    x_initial = x
    x = self.LN3(x).reshape(B,H,W,C)
    #shift
    x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
    # partition windows
    x_windows = window_partition(x, self.window_size)
    x_windows = x_windows.reshape(-1, self.window_size ** 2, C)
    # SW-MSA
    WindowAttention2 = self.WindowAttention2(x_windows, mask=self.attention_mask)
    # merge windows
    WindowAttention2 = WindowAttention2.reshape(-1, self.window_size, self.window_size, C)
    x = window_reverse(WindowAttention2, self.window_size, H, W)
    x = torch.roll(x, shifts=(self.shift_size, self.shift_size), dims=(1, 2))
    x = x.reshape(B, H * W, C)
    #FFN
    x = initial_x + self.drop_path(x)
    initial_x = x
    x = self.LN4(x)
    x = self.MLP2(x)
    x = self.drop_path(x)
    x += initial_x
    
    return x

class SwinTransformerModel(nn.Module):
  def __init__(self, img_size=224, patch_size=4, in_chans=3, num_classes=1000,
                 embed_dim=96, depths=[2, 2, 6, 2], num_heads=[3, 6, 12, 24],
                 window_size=7, mlp_ratio=4., qkv_bias=True, qk_scale=None,
                 drop_rate=0., attn_drop_rate=0., drop_path_rate=0.1,
                 norm_layer=nn.LayerNorm, ape=False, patch_norm=True,
                 use_checkpoint=False, **kwargs):
      super(SwinTransformerModel, self).__init__()
      self.patches_resolution = [img_size[0] // patch_size[0], img_size[1] // patch_size[1]]
      self.patch_size = patch_size
      self.embed_dim = embed_dim
      self.patch_embed = PatchEmbed()
      self.num_features = int(embed_dim * 2 ** (patch_size - 1))
      self.input_resolution = (patches_resolution[0], patches_resolution[1])
      self.block1 = SwinTransformerBlock(dim = int(embed_dim), input_resolution=(patches_resolution[0], patches_resolution[1]),num_heads[0])
      self.block2 = SwinTransformerBlock(dim = int(embed_dim * 2), input_resolution=(patches_resolution[0] // 2, patches_resolution[1] // 2), num_heads[1])
      self.block3 = SwinTransformerBlock(dim = int(embed_dim * 4), input_resolution=(patches_resolution[0] // 4, patches_resolution[1] // 4), num_heads[2])
      self.block4 = SwinTransformerBlock(dim = int(embed_dim * 8), input_resolution=(patches_resolution[0] // 8, patches_resolution[1] // 8), num_heads[3])
      self.patch_merging1 = PatchMerging()
      self.patch_merging2 = PatchMerging()
      self.patch_merging3 = PatchMerging()
      self.final_layer = nn.Linear(int(embed_dim * 8), num_classes)
      self.norm = nn.LayerNorm(self.num_features)
      self.avgpool = nn.AdaptiveAvgPool1d(1)
      self.head = nn.Linear(self.num_features, num_classes) if num_classes > 0 else nn.Identity()
  def forward(self, image):
    patch_embed = self.patch_embed(image, patch_size = self.patch_size, embed_dim)

    x = self.block1(patch_embedding)
    x = self.patch_merging1(x,self.input_resolution[0]//2,self.input_resolution[1]//2)
    x = self.block2(x)

    x = self.patch_merging2(x,self.input_resolution[0]//4,self.input_resolution[1]//4)
    x = self.block3(x)

    x = self.patch_merging3(x,self.input_resolution[0]//8,self.input_resolution[1]//8)
    x = self.block4(x)

    x = self.norm(x)
    x = self.avgpool(x.transpose(1, 2))
    x = torch.flatten(x, 1)
    x = self.head(x)
    return x

4. Data processing and training (Jiangyan Feng)